In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [2]:
processed_data=pd.read_csv('../data/processed_data.csv')

In [3]:
processed_data.head()

,ID,TSV,TPV,met,age,sex,w_st,a_st,HR,T_g,RH_in,T_in,CO_2,V_a,T_out,RH_out,clo,BMI,T_mrt
0,1,0,-1,1.2,30,Female,28.58200,28.7993,86.945652,18.29,39.77,18.63,732.9,0.038,1.336667,94.704930,1.35,22.832879,18.169260
1,2,-1,1,1.0,66,Male,29.90445,31.0381,74.120482,18.60,38.19,18.93,765.7,0.038,1.390000,94.639374,0.88,37.448347,18.483187
2,1,1,-1,1.2,30,Female,29.35660,30.2740,86.651685,18.63,38.02,18.96,763.3,0.039,1.396667,94.631180,1.35,22.832879,18.511387
3,2,-1,1,1.0,66,Male,29.95435,31.0880,73.465116,18.65,37.74,18.97,756.8,0.044,1.410000,94.614791,0.88,37.448347,18.526371
4,3,-1,1,1.0,47,Male,29.94800,32.5880,81.819149,19.35,32.49,18.60,641.1,0.035,1.423333,94.598402,0.89,22.351743,19.600292


In [4]:
X = processed_data.drop(columns=['TPV', 'TSV'])
y = processed_data['TPV']

X = pd.get_dummies(X, columns=['sex'], drop_first=True)
le = LabelEncoder()
X['ID'] = le.fit_transform(X['ID'])

Input missing values

In [5]:
imputer = KNNImputer(n_neighbors=5)
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

Correlation analysis to drop highly correlated features

In [6]:
corr_matrix = X_imputed.corr().abs()
upper = corr_matrix.where(~np.tril(np.ones(corr_matrix.shape)).astype(bool))
to_force_drop = ['T_mrt', 'T_g']  
to_keep = ['T_in']              

to_drop = [
    column for column in upper.columns
    if any(upper[column] > 0.95)
    and column not in to_keep  
    or column in to_force_drop  
]
X_filtered = X_imputed.drop(columns=to_drop)

to_drop

['T_g', 'T_mrt']